In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import tqdm

model_id = "MBZUAI/LaMini-GPT-124M"
peft_model_id = "FXNan/gpt2-124M-DPO"

model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

In [ ]:
dataset_id = "allenai/winogrande"

tokenizer = AutoTokenizer.from_pretrained(model_id)

ds = load_dataset(dataset_id, "winogrande_xs")

In [ ]:
ds_test = ds['validation']
print(ds_test)
print(ds_test['sentence'][0])
print(ds_test['option1'][0])
print(ds_test['option2'][0])
print(ds_test['answer'][0])

In [ ]:
questions = ds_test['sentence']
answers = []
for i in range(len(questions)):
    answer = ds_test['option1'][i] if ds_test['answer'][i] == 1 else ds_test['option2'][i]
    answers.append(answer)

In [ ]:
from string import Template
qa_template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n${q}\n\n### Response:"
qa_template = Template(qa_template)

model.eval()

loss_array = []

# for i in range(min(2000, len(questions))):
for i in tqdm.tqdm(range(min(100000, len(questions)))):
    q = questions[i]
    q = "fill in the blank: " + q
    a = answers[i]
    qa = qa_template.substitute(q=q)
    q_tokens = tokenizer(qa, return_tensors="pt", max_length=1024, truncation=True)
    q_tokens_len = len(q_tokens['input_ids'][0])
    tokens = tokenizer(qa+a, return_tensors="pt", max_length=1024, truncation=True)
    qa_tokens_len = len(tokens['input_ids'][0])
    a_tokens_len = qa_tokens_len - q_tokens_len
    if a_tokens_len < 1:
        print(f"Skipping {i}, qa_tokens_len: {qa_tokens_len}, q_tokens_len: {q_tokens_len}, a_tokens_len: {a_tokens_len}")
        print(f"q: {q}")
        print(f"a: {a}")
        continue
    
    with torch.no_grad():
        output = model(**tokens)
    
        logits = output['logits'][:, -a_tokens_len-1:-1, :]
        
        try:
            loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.shape[-1]), tokens['input_ids'][0, -a_tokens_len:].view(-1))
        except:
            print(f"Error at {i}")
            print(f"q_tokens_len: {q_tokens_len}")
            print(f"qa_tokens_len: {qa_tokens_len}")
        loss_array.append(loss.item())
        
        
print(f"Mean loss: {sum(loss_array)/len(loss_array)}")
        
# baseline: 3.988993604144856
# lora 5e-6: 3.984750234778462
# lora 1e-5: 3.93418066213003